In [ ]:
from babydragon.chat.memory_chat import FifoVectorChat, FifoChat, VectorChat
from babydragon.chat.base_chat import BaseChat
from babydragon.chat.chat import Chat
from babydragon.memory.indexes.pandas_index import PandasIndex
from babydragon.memory.indexes.python_index import PythonIndex
from babydragon.utils.oai import mark_question

In [ ]:
import openai
openai.api_key = ""

In [ ]:
import os
import babydragon

babydragon_path = os.path.dirname(os.path.abspath(babydragon.__file__))
print(babydragon_path)


In [ ]:
pind = PythonIndex(babydragon_path,name="babydragon_index", load = False)

In [ ]:
out, embed, ind = pind.faiss_query("Fifo_Threaded_Chat", k = 10)

In [ ]:
ic = Chat (model='gpt-3.5-turbo-0301',index_dict={"babydragon": pind}) 
vci = FifoVectorChat(model='gpt-3.5-turbo-0301',index_dict={"babydragon": pind}, max_output_tokens=500, max_index_memory= 1000, max_memory=2000, longterm_frac=0.3)

In [ ]:
#qa bot
# ic.gradio()

In [ ]:
#chatbot with memory ofconversation 
vci.gradio()